# Building a simple autoencoder

Based on the ability to generate UK car registration numbers, one can build a dataset of training and test data. With a suitable vectorizer, one can go ahead and make an autoencoder.

### References

https://blog.keras.io/building-autoencoders-in-keras.html

In [1]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegBowVectorizer

### Data generation

In [2]:
n_train = 10000
n_val = 1000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegBowVectorizer()

train_strs = [g.get_reg() for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
val_strs = [g.get_reg() for _ in range(n_val)]
val_strs = [x for x in val_strs if x not in set(train_strs)]  # sanity check
val_vecs = np.array([v.vectorize(x) for x in val_strs])
test_strs = [g.get_reg() for _ in range(n_test)]
test_strs = [x for x in test_strs if x not in set(train_strs) | set(val_strs)]  # sanity check
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print('input vector length = ' + str(vec_length))
print('actual number of validation samples = ' + str(len(val_strs)))
print('actual number of test samples = ' + str(len(test_strs)))

input vector length = 252
actual number of validation samples = 1000
actual number of test samples = 1000


### Standard autoencoder

Design the network. A single dense layer for the encoder and decoder needs an encoding dimension of around 100. Having more layers allows a smaller encoding dimension, although convergence during learning starts to become unstable.

In [3]:
encoding_dim = 30
input_reg = keras.Input(shape=(vec_length,))
# encoded = layers.Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_reg)
encoded = layers.Dense(vec_length, activation='relu')(input_reg)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)
encoder = keras.Model(input_reg, encoded)

encoded_input = keras.Input(shape=(encoding_dim,))
decoded = layers.Dense(encoding_dim, activation='relu')(encoded_input)
decoded = layers.Dense(vec_length, activation='relu')(decoded)
decoded = layers.Dense(vec_length, activation='sigmoid')(decoded)
decoder = keras.Model(encoded_input, decoded)

autoencoder = keras.Model(input_reg, decoder(encoder(input_reg)))

Do some learning. For shallow networks, almost any optimizer and loss work. For deeper networks, poor choices almost certainly yield failed convergence on the loss function. Even good choices yield convergence on the loss function only on some runs.

In [4]:
autoencoder.compile(optimizer='adam', loss='categorical_crossentropy')
autoencoder.fit(train_vecs, train_vecs,
                epochs=70,
                batch_size=256,
                shuffle=True,
                validation_data=(val_vecs, val_vecs))

Epoch 1/70
40/40 [==============================] - 1s 24ms/step - loss: 36.4166 - val_loss: 34.4567
Epoch 2/70
40/40 [==============================] - 1s 20ms/step - loss: 34.2345 - val_loss: 33.8965
Epoch 3/70
40/40 [==============================] - 1s 23ms/step - loss: 33.1213 - val_loss: 32.3815
Epoch 4/70
40/40 [==============================] - 1s 24ms/step - loss: 31.7734 - val_loss: 31.4186
Epoch 5/70
40/40 [==============================] - 1s 23ms/step - loss: 30.9189 - val_loss: 30.6846
Epoch 6/70
40/40 [==============================] - 1s 22ms/step - loss: 30.2002 - val_loss: 30.1389
Epoch 7/70
40/40 [==============================] - 1s 21ms/step - loss: 29.6775 - val_loss: 29.7219
Epoch 8/70
40/40 [==============================] - 1s 24ms/step - loss: 29.2640 - val_loss: 29.3271
Epoch 9/70
40/40 [==============================] - 1s 22ms/step - loss: 28.8387 - val_loss: 28.9637
Epoch 10/70
40/40 [==============================] - 1s 23ms/step - loss: 28.5153 - val_los

Evaluation

In [5]:
decoded_regs = autoencoder.predict(test_vecs)
print(test_strs[:10])
print([v.recover(x) for x in decoded_regs[:10]])
acc = np.sum([v.recover(x) == y for x, y in zip(decoded_regs, test_strs)]) / len(test_strs)
print('accuracy = ' + str(acc))

['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57ABA', 'CD77SJT']
accuracy = 0.857
